In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 6

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v02"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v02 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V06/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V06/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M07D31 UTC 09:34


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//155562_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//145102_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//122784_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//157807_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//142625_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//162203_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//172497_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//131902_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111011_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//232144_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//127443_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//223940_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//181976_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132895_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//232160_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115807_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111018_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//131909_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//122820_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//155571_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//127450_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//158116_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//145109_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//181993_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//224003_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//142634_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//162300_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132941_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132947_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//153034_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132170_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//145207_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//142642_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//184000_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//225111_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//158143_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//172542_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//122847_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//232250_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//116005_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111045_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//162444_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//145404_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132224_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111070_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//155616_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117101_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//172549_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//211016_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//127485_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//232303_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//153042_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//225118_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//162461_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//122855_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//142651_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117108_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//158218_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//127493_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//211034_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132973_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//122880_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//142658_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//162468_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//155634_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//232401_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//172558_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111088_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//132242_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//145602_V06.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//811820_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782665_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742422_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863012_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914406_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//753280_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772223_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//831880_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822450_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//811830_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863013_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914407_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742995_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782666_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742443_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914700_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//743008_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//831920_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//753401_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822472_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//811904_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772270_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863070_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782687_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914701_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//743009_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742444_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//831930_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911102_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822493_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914805_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//753503_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782699_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//831959_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//812107_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772344_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742483_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911103_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914806_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//751300_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822494_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//753504_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//812301_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772402_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911219_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//753625_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742495_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822517_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//832109_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782804_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//812302_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911220_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772403_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//753626_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//742496_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822518_V06.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//832443_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911285_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782991_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822560_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772608_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//812472_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//832444_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911286_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//782992_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//822570_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//832445_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//772609_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//812473_V06.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911287_V06.csv
/volumes/data/Y2018M

In [14]:
#!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111011_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111012_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111013_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111014_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111015_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111016_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111017_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111030_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//111018_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112140_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112160_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112150_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112180_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112190_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112200_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112300_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112400_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//112700_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114443_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114444_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114445_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114446_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114447_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114448_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114449_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114450_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//114460_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115423_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115424_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115425_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115426_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115427_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115428_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115429_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115430_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//115440_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117101_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117102_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117103_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117104_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117105_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117106_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117107_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117108_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//117109_V06.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863000_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863001_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863002_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863003_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863004_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863005_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863006_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863007_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//863008_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911287_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911288_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911289_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911290_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911300_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911401_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911402_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911403_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//911404_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912108_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912109_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912201_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912202_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912203_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912204_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912205_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912206_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//912207_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913442_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913443_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913444_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913445_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913446_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913447_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913448_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913449_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//913450_V06.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914401_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914402_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914403_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914404_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914405_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914406_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914407_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914408_V06.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V06//914409_V06.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:42:01.416597


Previous runs:  
0:42:01.416597
